In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import csv

In [149]:
train_data = pd.read_csv("./train.csv")

In [3]:
train_data.head()

,ID,Score,col1,col2,col3,col4,col5,col6,col7,col8,...,col3796,col3797,col3798,col3799,col3800,col3801,col3802,col3803,col3804,col3805
0,5,3.475628,0,4.058,0.824,0,10.267,0.728,4.403,0.050,...,1.067,0,0.0,0.115,30.395,24.541,0,0.415,0.997,0
1,8,3.601332,0,4.111,0.929,0,8.352,0.907,4.216,0.034,...,0.934,0,0.0,0.227,38.508,35.038,0,3.979,0.997,3
2,9,1.935003,0,4.139,0.833,66,9.494,0.733,4.069,0.267,...,1.722,0,0.0,0.148,27.932,19.518,0,0.849,0.999,0
3,12,3.283663,0,4.016,0.880,0,8.237,0.836,3.956,0.129,...,0.993,0,0.0,0.124,18.993,25.403,0,0.988,0.998,0
4,14,3.409121,0,4.657,0.522,0,35.882,0.383,4.234,-0.089,...,2.095,0,0.0,0.088,44.225,15.741,0,1.595,0.997,0


In [88]:
null_count = 0
for s in train_data.isnull().sum().values:
    if s > 0:
        null_count += 1
if null_count > 0:
    print("null")
else:
    print("no null")

no null


In [127]:
corr_list = list()
delete_list = list()
for i in range(3804):
    corr = np.corrcoef(y, train_data["col" + str(i+1)].values)[0][1]
    
    if np.isnan(corr):
        delete_list.append("col{}".format(i+1))
    else:
        corr_list.append(corr)

In [150]:
train_data = train_data.drop(delete_list, axis=1)

In [151]:
train_data.shape

(13731, 3753)

In [152]:
# 説明変数
X = train_data.drop(["ID", "Score"], axis=1).values.astype("int")
# 目的変数
y = train_data["Score"].values.astype("int")

# 訓練データとテストデータに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6, random_state=5)

In [154]:
from sklearn import svm

model = SVC(gamma=0.01, C=100.)
model.fit(X_train, y_train)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=0.01, kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [155]:
# 平均二乗誤差
from sklearn.metrics import mean_squared_error
#  精度
from sklearn.metrics import accuracy_score


# 目的変数を予測
y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

# train の誤差が test よりも圧倒的に小さい場合は過学習
print("train_data : {}".format(mean_squared_error(y_train, y_train_predict)))
print("test_data  : {}".format(mean_squared_error(y_test, y_test_predict)))

accuracy_score(y_train_predict, y_train)

train_data : 0.0
test_data  : 1.0784075737346766


1.0

In [136]:
test_data = pd.read_csv("test.csv")
test_id = test_data["ID"].values
test_data = test_data.drop("ID", axis=1)
test_data = test_data.drop(delete_list, axis=1)

test_datas = test_data.values.astype("int")

In [137]:
predict_data = model.predict(test_datas)

In [138]:
predict_csv = list()
for index, id in enumerate(test_id):
    tmp_list = list()
    tmp_list.append(id)
    tmp_list.append(predict_data[index])
    predict_csv.append(tmp_list)

# csv 出力
with open("predict.csv", "w", encoding="Shift_jis") as f:
    writer = csv.writer(f, lineterminator="\n")
    writer.writerows(predict_csv)